In [4]:
import os
import cv2
import base64
import requests
import pickle

In [45]:
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
import pickle
import xgboost as xgb
from sklearn.ensemble import VotingClassifier

In [40]:
path = "CarsDataset/test" # Cars Dataset/train & Cats Dataset/test ใช้สำหรับสับ path ของ train และ test
list_x = [] #เก็บ Base64 
list_y = [] #เก็บไฟล์เกี่ยวกับรถยนต์

for sub in os.listdir(path): #นำรูปภาพจาก path มาเก็บโดย loop มาเก็บ
    for fn in os.listdir(os.path.join(path, sub)):
        path_file_img = os.path.join(path, sub, fn)
        readImage = cv2.imread(path_file_img, cv2.IMREAD_GRAYSCALE)
        list_x.append(readImage) #เก็บ Base64 
        list_y.append(sub) #เก็บไฟล์เกี่ยวกับรถยนต์

In [41]:
def imgtovec(img): # Funtion สำหรับเก็บ Api
    try:
        resized_img = cv2.resize(img, (128, 128), cv2.INTER_AREA)
        v, buffer = cv2.imencode(".jpg", resized_img)
        img_str = base64.b64encode(buffer).decode('utf-8')
        image_data_string = img_str

        url = "http://127.0.0.1:80/api/genhog"
        params = {"img": image_data_string}

        response = requests.get(url, json=params)

        if response.status_code == 200:
            return response.json()
        else:
            return {"error": f"เรียก API ไม่สำเร็จ API CODE : {response.status_code}"}
    except Exception as ex:
        return {"error": f"เกิดข้อผิดพลาด: {str(ex)}"}

In [42]:
hogvectors = [] # ส่งรูป base64 ให้ api แปลงเป็น hogvector #
for i in range(len(list_x)):
    res = imgtovec(list_x[i])
    vec = list(res["hog"])
    vec.append(list_y[i])
    hogvectors.append(vec)

In [46]:
write_path = "hogvectors_test.pkl" #swap train to test สร้างไฟล์สำหรับ test และ train 
pickle.dump(hogvectors, open(write_path, "wb"))
print("data preparation is done")

data preparation is done


In [69]:
hogvectors_train = pickle.load(open('hogvectors_train.pkl', 'rb')) #อ่านไฟล์มาเก็บสำหรับ train
hogvectors_test = pickle.load(open('hogvectors_test.pkl', 'rb')) #อ่านไฟล์มาเก็บสำหรับ train

In [70]:
# ดึงข้อมูลแถวและคอลัมน์มาทำเป็น feature และเก็บค่าไว้แยกกันสำหรับ train และ test 
X_train_data = [hogfeature_Xtrain[0:8100] for hogfeature_Xtrain in hogvectors_train] 
X_test_data = [hogfeature_Xtest[0:8100] for hogfeature_Xtest in hogvectors_test]

In [71]:
# ดึงข้อมูลแถวและคอลัมน์มาทำเป็น class และเก็บค่าไว้แยกกันสำหรับ train และ test 
Y_train_data = [hogfeature_Ytrain[-1] for hogfeature_Ytrain in hogvectors_train]
Y_test_data = [hogfeature_Ytest[-1] for hogfeature_Ytest in hogvectors_test]

In [72]:
label_encoder = LabelEncoder() #object label_encoder จาก Class LabelEncoder แปลงชื่อยี่ห้องมาเป็นข้อมูลตัวเลข
y_cls_train = label_encoder.fit(Y_train_data) # .fit(Y_train_data) เพื่อ train ชื่อยี่ห้อรถยนต์จะถูกแทนด้วยตัวเลข 
y_labelNum_train = label_encoder.transform(Y_train_data)
y_cls_test = label_encoder.fit(Y_test_data)
y_labelNum_test = label_encoder.transform(Y_test_data) 
# แปลง String ยี่ห้อรถจาก Test เป็น TableIndex

In [73]:
clf = DecisionTreeClassifier(random_state=42)
# สร้าง object model DecisionTree

In [74]:
xgb_model = xgb.XGBClassifier(objective="multi:softmax",num_class=len(label_encoder.classes_), random_state=42)
# สร้าง object modelXGBoost

In [75]:
ensemble_model = VotingClassifier(estimators=[('DecisionTree', clf), ('XGBoost', xgb_model)], voting='hard',weights=[1, 4])
# ทำการรวม 2 model และเก็บไว้ 

In [76]:
ensemble_model.fit(X_train_data, y_labelNum_train)
# .fit() X_train_data เป็นการให้ model train ข้อมูล และ Y_train_data เป็นคำตอบที่ควรจะได้จากการ train

VotingClassifier(estimators=[('DecisionTree',
                              DecisionTreeClassifier(random_state=42)),
                             ('XGBoost',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None, gamma=None,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=N

In [77]:
y_pred = ensemble_model.predict(X_test_data) 
# ทดสอบ X_test_data เพื่อไว้ทำนายผลลัพธ์ที่ได้จากโมเดลนี้ของข้อมูล test

In [78]:
accuracy = accuracy_score(y_labelNum_test, y_pred)
confusionMatrix = confusion_matrix(y_labelNum_test, y_pred)
print("Accuracy: ", accuracy)
print("Confusion Matrix: ", confusionMatrix) 
#นำค่า Accuracy ที่ได้จากการ test มาแสดง ยิ่งใกล้ 1 ยิ่งแม่นยำ

Accuracy:  0.6629766297662977
Confusion Matrix:  [[167   1   2   4   1   9  15]
 [  7  41   0   2   3   2  12]
 [ 12   1  36   0   2   1  23]
 [ 41   1   3  19   1   2   7]
 [ 18   4   3   0  49   3  25]
 [ 19   3   2   0   2  74   6]
 [ 21   1   0   2  10   3 153]]


In [79]:
path_model = 'model_genhog.pkl'
pickle.dump(ensemble_model, open(path_model, 'wb'))
# สร้าง model_genhog.pkl เพื่อทำการเก็บ Model การ train and test ครั้งนี้